In [1]:
!pip -q install -U aiogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.3/677.3 kB 11.0 MB/s eta 0:00:00


In [2]:
!pip -q install azure-ai-inference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 10.1 MB/s eta 0:00:00


In [3]:
import asyncio
import os
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
import logging
import sys

In [ ]:
# Retrieve environment variables from Google Colab userdata
from google.colab import userdata
os.environ['GITHUB_TOKEN'] = ''
os.environ['TELEGRAM_BOT_TOKEN'] =''

In [5]:
dp = Dispatcher()

In [6]:
# Context class to store previous responses (basic implementation)
class Gpt_context:
    def __init__(self):
        self.response = ""

# Create context instance
context = Gpt_context()

In [7]:
# Handler for /start command
@dp.message(Command('start'))
async def command_start_handler(message: types.Message):
    """Handles the /start command"""
    await message.answer("Hello sir!")

In [8]:
# Handler for /help command
@dp.message(Command('help'))
async def command_help_handler(message: types.Message):
    """Handles the /help command"""
    help_command = """
    Helo sir U can chose from -
    /start
    /help
    Send text, audio, or image for processing!
    """
    await message.reply(help_command)

In [ ]:
# Handler for text messages
@dp.message()
async def chatgpt(message: types.Message):
    """Processes text messages using Azure AI Inference (GitHub model)"""
    endpoint = "https://models.github.ai/inference"
    model = "openai/gpt-4.1"
    token = ""

    client = ChatCompletionsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token),
    )

    try:
        response = client.complete(
            messages=[
                SystemMessage(content="You are a helpful assistant."),
                UserMessage(content=message.text),
            ],
            temperature=1.0,
            top_p=1.0,
            model=model
        )
        value = response.choices[0].message.content
        context.response = value  # Store response in context
        print("Bot Response:", value)
        await message.reply(value)
    except Exception as e:
        await message.reply(f"Error: {str(e)}")

In [ ]:
# Main function to start the bot
async def main() -> None:
    bot = Bot('')
    await dp.start_polling(bot)

In [11]:
# Function to run the bot in Colab's event loop
async def run_bot():
    try:
        await main()
    except KeyboardInterrupt:
        print("Bot stopped manually")

In [12]:
#Run the bot using the existing event loop
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, stream=sys.stdout)
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # If the loop is already running, schedule the coroutine
        asyncio.ensure_future(run_bot())
    else:
        # If the loop is not running, start it
        try:
            loop.run_until_complete(run_bot())
        finally:
            loop.run_until_complete(loop.shutdown_asyncgens())
            loop.close()